In [1]:
import re
import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import os

import re

import ujson

import json

import pandas as pd

from pathlib import Path


In [2]:

# Ruta de la carpeta de datos de Google Maps
google_maps_folder = 'data/Google Maps/'


In [3]:
google_maps_categories = []

# Cargar las categorías de Google Maps
for filename in os.listdir(os.path.join(google_maps_folder, 'metadata-sitios')):
    if filename.endswith('.json'):
        with open(os.path.join(google_maps_folder, 'metadata-sitios', filename)) as file:
            for line in file:
                try:
                    data = json.loads(line)
                    categories = data.get('category')
                    if categories:
                        google_maps_categories.extend(categories)
                except json.JSONDecodeError as e:
                    print(f"Error al cargar el archivo JSON: {filename}")
                    print(e)




In [4]:
# Obtener los distintos tipos de bares
unique_bar_types = set()
for category in google_maps_categories:
    if re.search(r'\bbar\b', category, re.IGNORECASE):
        match = re.search(r'\b(bar)\b', category, re.IGNORECASE)
        if match:
            bar_type = category[:match.start()].strip()
            unique_bar_types.add(bar_type)

In [5]:
# Ruta de la carpeta principal
carpeta_principal = 'data/Google Maps'

# Ruta de la carpeta de subcarpetas
subcarpeta = os.path.join(carpeta_principal, 'metadata-sitios')


In [6]:
# Lista para almacenar los resultados filtrados
resultados = []

# Recorrer todos los archivos JSON dentro de la carpeta "metadata-sitios"
for archivo in os.listdir(subcarpeta):
    if archivo.endswith(".json"):
        # Ruta del archivo JSON actual
        ruta_archivo = os.path.join(subcarpeta, archivo)

        # Leer el contenido del archivo JSON
        with open(ruta_archivo) as f:
            contenido_json = f.readlines()

        # Cargar cada objeto JSON por separado
        for linea in contenido_json:
            try:
                data = json.loads(linea)
            except json.JSONDecodeError as e:
                print(f"Error al cargar el archivo JSON {ruta_archivo}: {e}")
                continue

            # Verificar si la categoría "bares" está presente en el archivo JSON
            categorias = data.get("category", [])
            if isinstance(categorias, list) and any("bar" in categoria.lower() for categoria in categorias):
                resultados.append(data)

# Convertir los resultados a un pandas DataFrame
df = pd.DataFrame(resultados)

# Convertir el pandas DataFrame a una tabla PyArrow
table = pa.Table.from_pandas(df)

# Ruta del archivo de salida en formato Parquet
ruta_salida = os.path.join(carpeta_principal, "resultados.parquet")

# Guardar los resultados en el archivo Parquet de salida
pq.write_table(table, ruta_salida)


# Lista para almacenar los resultados filtrados
resultados = []


# Recorrer todos los archivos JSON dentro de la carpeta "metadata-sitios"
for archivo in os.listdir(subcarpeta):
    if archivo.endswith(".json"):
        # Ruta del archivo JSON actual
        ruta_archivo = os.path.join(subcarpeta, archivo)
        
        # Leer el contenido del archivo JSON
        with open(ruta_archivo) as f:
            contenido_json = f.readlines()
        
        # Cargar cada objeto JSON por separado
        for linea in contenido_json:
            try:
                data = json.loads(linea)
            except json.JSONDecodeError as e:
                print(f"Error al cargar el archivo JSON {ruta_archivo}: {e}")
                continue
            
            # Verificar si la categoría "bares" está presente en el archivo JSON
            categorias = data.get("category", [])
            if isinstance(categorias, list) and any("bar" in categoria.lower() for categoria in categorias):
                resultados.append(data)

# Ruta del archivo de salida
ruta_salida = os.path.join(carpeta_principal, "resultados.json")

# Guardar los resultados en el archivo JSON de salida
with open(ruta_salida, "w") as f:
    json.dump(resultados, f)